In [1]:
import torch
import matplotlib.pyplot as plt
import cv2
import joblib
import quaternion as q
import numpy as np
from src.models.autoencoder.autoenc import Embedder
from src.utils.render_utils import add_agent_view_on_w, add_title
import imageio
from IPython.display import HTML, display
import copy
from src.utils.camera_trajectory import go_forward, go_backward, rotate_n

In [2]:
import os
os.environ['GLOG_minloglevel'] = "3"
os.environ['MAGNUM_LOG'] = "quiet"
os.environ['HABITAT_SIM_LOG'] = "quiet"
import habitat
from habitat import get_config
from habitat.sims import make_sim

config = get_config()
cfg = config
cfg.defrost()
habitat_api_path = os.path.join(os.path.dirname(habitat.__file__), '../')
cfg.DATASET.SCENES_DIR = os.path.join(habitat_api_path, cfg.DATASET.SCENES_DIR)
cfg.DATASET.DATA_PATH = os.path.join(habitat_api_path, cfg.DATASET.DATA_PATH.replace("habitat-test-scenes","gibson"))
cfg.SIMULATOR.SCENE_DATASET = 'data/scene_datasets/mp3d.scene_dataset_config.json'
cfg.SIMULATOR.SCENE_DATASET = os.path.join(habitat_api_path, cfg.SIMULATOR.SCENE_DATASET )
cfg.DATASET.TYPE = "PointNav-v1"
cfg.SIMULATOR.RGB_SENSOR.HEIGHT = 128
cfg.SIMULATOR.RGB_SENSOR.WIDTH = 128
cfg.SIMULATOR.DEPTH_SENSOR.HEIGHT = 128
cfg.SIMULATOR.DEPTH_SENSOR.WIDTH = 128
cfg.TASK.SENSORS = cfg.SIMULATOR.AGENT_0.SENSORS = ['RGB_SENSOR', 'DEPTH_SENSOR']
cfg.SIMULATOR.AGENT_0.HEIGHT = 1.25
cfg.freeze()

from habitat.datasets import make_dataset
dataset = make_dataset('PointNav-v1')
cfg.defrost()
cfg.DATASET.TYPE = 'PointNav-v1'
cfg.DATASET.SPLIT = 'val'
cfg.freeze()
all_scenes = dataset.get_scenes_to_load(cfg.DATASET)

2023-06-09 13:39:23,114 Initializing dataset PointNav-v1


In [3]:
device = 'cuda'
embedder = Embedder(pretrained_ckpt='pretrained/autoenc_large.ckpt',
                   img_res=128, w_size=128, coordinate_scale=32, w_ch=32, nerf_res=128, voxel_res=128)
embedder = embedder.to(device).eval()

In [4]:
scene = 'Cantwell'#np.random.choice(all_scenes)
print(scene)

Cantwell


In [5]:
try: sim.close()
except: pass
cfg.defrost()
cfg.SIMULATOR.SCENE = os.path.join(cfg.DATASET.SCENES_DIR,'gibson_habitat/{}.glb'.format(scene))
cfg.freeze()
past_room = scene
sim = make_sim(id_sim=cfg.SIMULATOR.TYPE, config=cfg.SIMULATOR)

2023-06-09 13:39:27,006 initializing sim Sim-v0


## Generate and Embed Random Trajectory

In [7]:
NUM_GOALS = 5
from habitat.tasks.nav.shortest_path_follower import ShortestPathFollower

follower = ShortestPathFollower(sim, 0.5, return_one_hot=False)

start_position = sim.sample_navigable_point()
start_rotation = q.from_euler_angles([0., np.random.rand() * 2 * np.pi, 0.])

orig_Rt = np.eye(4)
orig_Rt[:3,3] = start_position
orig_Rt[:3,:3] = q.as_rotation_matrix(start_rotation)
orig_Rt = np.linalg.inv(orig_Rt)

goal_points = []
for i in range(NUM_GOALS):
    while True:
        random_point = sim.sample_navigable_point()
        if abs(random_point[1]-start_position[1]) > 0.5: continue
        break
    goal_points.append(random_point)

sim.set_agent_state(start_position, start_rotation)
done = False
goal_point = goal_points.pop(0)

K = torch.eye(3)
K[0,0] = (embedder.img_res/2.) / np.tan(np.deg2rad(90.0) / 2)
K[1,1] = -(embedder.img_res/2.) / np.tan(np.deg2rad(90.0) / 2)
K = K.unsqueeze(0).to(device)

w, w_mask = None, None
imgs = []
while not done:
    state = sim.get_agent_state()
    obs = sim.get_observations_at(state.position, state.rotation)

    Rt_t = np.eye(4)
    Rt_t[:3,3] = state.position
    Rt_t[:3,:3] = q.as_rotation_matrix(state.rotation)
    Rt_t = np.linalg.inv(Rt_t)
    Rt_t = Rt_t @ np.linalg.inv(orig_Rt)

    rgb_t = torch.from_numpy(obs['rgb']/255.).unsqueeze(0).permute(0,3,1,2).to(device)
    depth_t = torch.from_numpy(obs['depth']).unsqueeze(0).permute(0,3,1,2).to(device)
    Rt_t = torch.from_numpy(Rt_t).unsqueeze(0).float().to(device)

    with torch.no_grad():
        output = embedder.calculate_mask_func(depth_t*10.0, Rt_t, K)
        sorted_indices, seq_unique_list, seq_unique_counts, _ = output
        input_dict = {'rgb': rgb_t.unsqueeze(1),
                      'depth': depth_t.unsqueeze(1),
                    'sorted_indices': sorted_indices.unsqueeze(1),
                    'seq_unique_counts': seq_unique_counts.unsqueeze(1),
                      'seq_unique_list': seq_unique_list.unsqueeze(1)}
        w, w_mask = embedder.embed_obs(input_dict, past_w=w, past_w_num_mask=w_mask)
        recon_rgb, _ = embedder.generate(w, {'Rt': Rt_t.unsqueeze(1), 'K':K.unsqueeze(1)}, out_res=64)

        orig_rgb = add_title(obs['rgb'], 'observation')
        recon_rgb = (recon_rgb.squeeze().permute(1,2,0).detach().cpu() * 255).numpy().astype(np.uint8)
        recon_rgb = cv2.resize(recon_rgb, (orig_rgb.shape[0], orig_rgb.shape[1]))
        recon_rgb = add_title(recon_rgb, 'recon obs.')

        w_im = w.mean(0).mean(0).detach().cpu().numpy()
        w_im = ((w_im - w_im.min())/(w_im.max()-w_im.min()) * 255).astype(np.uint8)
        w_im = cv2.applyColorMap(w_im, cv2.COLORMAP_VIRIDIS)[:,:,::-1]
        last_w_im = w_im
        w_im = add_agent_view_on_w(w_im, Rt_t, embedder.coordinate_scale, embedder.w_size, agent_size=4, view_size=15)
        w_im = np.fliplr(w_im)
        w_im = add_title(w_im, 'RNR-map')

    view_im = np.concatenate([orig_rgb, recon_rgb, w_im],1)
    imgs.append(view_im)

    cv2.imshow("view", view_im[:,:,::-1])
    key = cv2.waitKey(1)
    if key == ord("q"): break



    action = follower.get_next_action(goal_point)
    if action is None or action == 0:
        if len(goal_points) == 0:
            break
        goal_point = goal_points.pop(0)
        action = follower.get_next_action(goal_point)
    sim.step(action)

last_w = w

In [8]:
imageio.mimwrite('demo/embedding_traj_from_simulator.gif', imgs, fps=15)
display(HTML('<img src={}>'.format("demo/embedding_traj_from_simulator.gif")))

## Explore inside RNR-Map
- Press 'w, a, s, d' to move
- Press 'q' to quit

In [9]:
images = []
Rt_current = torch.eye(4).unsqueeze(0).to(device).unsqueeze(1)
while True:
    with torch.no_grad():
        rgb, _ = embedder.generate(last_w, {"Rt": Rt_current, 'K': K.unsqueeze(1)}, out_res=64)
        rgb = (rgb.squeeze().permute(1,2,0).detach().cpu() * 255).numpy().astype(np.uint8)
        rgb = cv2.resize(rgb, (obs['rgb'][0].shape[0], obs['rgb'][0].shape[0]))
        rgb = add_title(rgb, 'Recon Image')
        w_color = copy.deepcopy(last_w_im)
        w_color = add_agent_view_on_w(w_color, Rt_current, embedder.coordinate_scale, embedder.w_size, agent_size=4, view_size=15)
        w_color = np.fliplr(w_color)
        w_color = add_title(w_color, 'Map')

        sim_Rt = np.linalg.inv(Rt_current.squeeze().detach().cpu().numpy() @ orig_Rt)
        position = sim_Rt[:3,3]
        rotation = sim_Rt[:3,:3]
        gt_obs = sim.get_observations_at(position, q.from_rotation_matrix(rotation))
        gt_rgb = add_title(gt_obs['rgb'], 'GT Image')

        view_im = np.concatenate([gt_rgb, rgb, w_color],1)
        cv2.imshow("view", view_im[:,:,::-1])
        key = cv2.waitKey(0)
        if key == ord('q'): break
        elif key == ord('a'):
            Rt = rotate_n(n=-10.0).to(device)
            Rt_current = (Rt@Rt_current.squeeze()).unsqueeze(0).unsqueeze(0)
        elif key == ord('d'):
            Rt = rotate_n(n=10.0).to(device)
            Rt_current = (Rt@Rt_current.squeeze()).unsqueeze(0).unsqueeze(0)
        elif key == ord("w"):
            Rt_current = go_forward(Rt_current, step=0.1)
        elif key == ord('s'):
            Rt_current = go_backward(Rt_current, step=0.1)
        images.append(view_im)

In [10]:
imageio.mimwrite("demo/explore_RNR_map_from_simluator.gif", images, fps=15)
display(HTML('<img src={}>'.format("demo/explore_RNR_map_from_simluator.gif")))